# Install fission

With helm, grafana, opentelemetry, prometheus, and jaeger enabled.

Install cert-manager and open-telemetry operator.

In [ ]:
# cert-manager
!kubectl apply -f https://github.com/jetstack/cert-manager/releases/latest/download/cert-manager.yaml

In [ ]:
# open telemetry operator
!kubectl apply -f https://github.com/open-telemetry/opentelemetry-operator/releases/latest/download/opentelemetry-operator.yaml

Create opentelemetry operator:

In [ ]:
%%bash
kubectl apply -f ./opentelemetry.yaml


Create jaeger namespace:

In [ ]:
%%bash
kubectl create namespace observability
kubectl create -n observability -f https://github.com/jaegertracing/jaeger-operator/releases/download/v1.39.0/jaeger-operator.yaml

Create jaeger instance:

In [ ]:
%%bash
kubectl apply -n observability -f - <<EOF
apiVersion: jaegertracing.io/v1
kind: Jaeger
metadata:
  name: jaeger
EOF

Port forward jaeger service:

In [ ]:
# use 31931 port
import subprocess

subprocess.run("kubectl port-forward service/jaeger-query -n observability 31931:16686 &", shell=True)

Install fission

In [ ]:
%%bash
export FISSION_NAMESPACE=fission
kubectl create -k "github.com/fission/fission/crds/v1?ref=v1.20.1"
kubectl create namespace $FISSION_NAMESPACE
helm install --namespace $FISSION_NAMESPACE \
  fission fission-charts/fission-all \
  --set openTelemetry.otlpCollectorEndpoint="otel-collector.opentelemetry-operator-system.svc:4317" \
  --set openTelemetry.otlpInsecure=true \
  --set openTelemetry.tracesSampler="parentbased_traceidratio" \
  --set openTelemetry.tracesSamplingRate="1"

Create grafana namespace:

In [ ]:
%%bash
export METRICS_NAMESPACE=monitoring
kubectl create namespace $METRICS_NAMESPACE

Install grafana:

In [ ]:
%%bash
helm repo add prometheus-community https://prometheus-community.github.io/helm-charts
helm repo update
helm install prometheus prometheus-community/kube-prometheus-stack -n monitoring

Upgrade fission using helm:

In [ ]:
!helm upgrade fission fission-charts/fission-all --namespace fission -f values.yaml

Expose grafana service:

In [ ]:
import subprocess

subprocess.run("kubectl --namespace monitoring port-forward svc/prometheus-grafana 39131:80 &",shell=True)

Get grafana password:

In [ ]:
%%bash
kubectl get secret --namespace monitoring prometheus-grafana -o jsonpath="{.data.admin-password}" | base64 --decode ; echo